# Coupled Spring System solve with initial conditions 

![image](images/Mass-spring-model-of-a-2-DOF-system-consisting-of-two-coupled-resonators.png)

$$
L = \frac{1}{2} m_1 \frac{d^2x_1}{dt^2} + \frac{1}{2} m_2 \frac{d^2x_2}{dt^2} - \frac{1}{2}\left(k_l x_1^2 + k_r x_2^2 + k_m (x_1 - x_2)^2\right)
$$

1. For the first mass:
$$
\frac{d}{dt} \left( \frac{dL}{dx_1} \right) = -k_lx_1 - k_m(x_1 - x_2)
$$

2. And for the second mass:
$$
\frac{d}{dt} \left( \frac{dL}{dx_2} \right) = -k_rx_2 + k_m(x_1 - x_2)
$$

so the coupled differential equations are 
$$
m_1 \frac{d^2x_1}{dt^2} + (k_l + k_m) x_1 - k_m x_2 = 0
$$
and 
$$
m_2 \frac{d^2x_2}{dt^2} + (k_r + k_m) x_2 - k_m x_1 = 0
$$

let $ m_1 = 1, m_2 = 2, k_l = 0.5, k_r = 0.75, k_m = 1$
so the equations become, 
$$
\frac{d^2x_1}{dt^2} = -1.5x_1 + x_2
$$
$$
\frac{d^2x_2}{dt^2} = -0.875x_2 + x_1
$$
We now proceed to solve the differential equations for various boundary conditions and assume 
$ -1 \leq x_{1} \leq 2$ and $ -2 \leq x_{2} \leq 1$ such that $L = 3$ and $ -1 \leq v_{1} \leq 1$, $ -1 \leq v_{2} \leq 1$

In [4]:
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
from scipy.integrate import solve_ivp

class SimpleFFN(nn.Module):
    """
    Defines a simple feedforward neural network (FFN) with one hidden layer.
    - N_INPUT: Number of input features.
    - N_OUTPUT: Number of output features.
    - N_HIDDEN: Number of neurons in the hidden layer.
    """
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN):
        super().__init__()
        self.activation = nn.Tanh()
        self.input_layer = nn.Linear(N_INPUT, N_HIDDEN)
        self.output_layer = nn.Linear(N_HIDDEN, N_OUTPUT)
    def forward(self, x):
        x = self.activation(self.input_layer(x))        
        x = self.output_layer(x)
        return x
# Example instantiation of the model

def differential_equations(t, y):
    x1, x2, v1, v2 = y
    dx1dt = v1
    dx2dt = v2
    dv1dt = -1.5*x1 + x2
    dv2dt = -0.875*x2 + x1
    return [dx1dt, dx2dt, dv1dt, dv2dt]

def solve_ivps(initial_conditions, t_span=[0, 1], t_eval=np.linspace(0, 1, 1000)):
    sol = solve_ivp(differential_equations, t_span, initial_conditions, t_eval=t_eval, method='RK45')
    return sol.y.T, sol.t

def initial_conditions(batch_size=32):
    x1_0 = np.random.uniform(-1, 2, (batch_size, 1))
    x2_0 = np.random.uniform(-2, 1, (batch_size, 1))
    v1_0 = np.random.uniform(-1, 1, (batch_size, 1))
    v2_0 = np.random.uniform(-1, 1, (batch_size, 1))
    return np.hstack((x1_0, x2_0, v1_0, v2_0))

torch.manual_seed(123)
model = SimpleFFN(5, 2, 10)  
optimizer = Adam(model.parameters())

for epoch in range(100000):
    optimizer.zero_grad()
    initials = initial_conditions(batch_size=1)  # For simplicity, batch_size=1
    loss = 0
    
    for ic in initials:
        ic_flat = ic.flatten()
        states, t_eval_np = solve_ivps(ic_flat)
        states = torch.tensor(states, dtype=torch.float32)
        t_eval = torch.tensor(t_eval_np, dtype=torch.float32, requires_grad=True).view(-1, 1)  # This is crucial
        ic_tensor = torch.tensor(ic, dtype=torch.float32)
        model_input = torch.cat((t_eval, ic_tensor.repeat(len(t_eval), 1)), 1)
        
        x_pred = model(model_input)
        
        # Physics-informed loss: derivatives
        x1_pred, x2_pred = x_pred[:, 0], x_pred[:, 1]
        v1_pred = torch.autograd.grad(x1_pred.sum(), t_eval, create_graph=True)[0]
        v2_pred = torch.autograd.grad(x2_pred.sum(), t_eval, create_graph=True)[0]
        a1_pred = torch.autograd.grad(v1_pred.sum(), t_eval, create_graph=True)[0]
        a2_pred = torch.autograd.grad(v2_pred.sum(), t_eval, create_graph=True)[0]
        
        # Loss
        physics_loss = torch.mean((a1_pred + 1.5*x1_pred - x2_pred)**2) + torch.mean((a2_pred + 0.875*x2_pred - x1_pred)**2)
        mse_loss = torch.mean((x_pred - states[:, :2])**2)        
        total_loss = mse_loss + physics_loss
        loss += total_loss
    
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


Epoch 0, Loss: 0.05306294932961464
Epoch 1000, Loss: 0.022910848259925842
Epoch 2000, Loss: 0.4846821129322052
Epoch 3000, Loss: 0.19462203979492188
Epoch 4000, Loss: 0.22618910670280457
Epoch 5000, Loss: 0.01177764218300581
Epoch 6000, Loss: 0.13835124671459198
Epoch 7000, Loss: 0.018267080187797546
Epoch 8000, Loss: 0.2298504263162613
Epoch 9000, Loss: 0.1902030110359192
Epoch 10000, Loss: 0.10836674273014069
Epoch 11000, Loss: 0.10459430515766144
Epoch 12000, Loss: 0.14901548624038696
Epoch 13000, Loss: 0.05463700741529465
Epoch 14000, Loss: 0.03009019047021866
Epoch 15000, Loss: 0.17116054892539978
Epoch 16000, Loss: 0.06472314894199371


KeyboardInterrupt: 